In [28]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("TU_CLAVE_SECRETA_2")

In [29]:
from openai import OpenAI  
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi
import yt_dlp

In [30]:
# Función para procesar cada URL y obtener la información del video
def summary_video_yt(youtube_url):
    try:
        # Obtener información del video con yt_dlp
        ydl_opts = {}
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=False)

        video_info = {
            "Titulo": info.get("title"),
            "Autor": info.get("uploader"),
            "Duracion": info.get("duration"),
            "Fecha_Publicacion": info.get("upload_date"),
            "URL": youtube_url
        }

        # Obtener el ID del video
        video_id = info.get("id")

        # Obtener transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['es', 'en'])
        transcript_text = " ".join([entry['text'] for entry in transcript])

        # Agregar el transcript al diccionario
        video_info["Transcript"] = transcript_text

        # Conectamos con OpenAI
        client = OpenAI(api_key=api_key)

        respuesta = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{
                "role": "user",
                "content": f"""Un resumen de la siguiente transcript,
                sabiendo que el titulo es: '{video_info["Titulo"]}',
                el autor es: '{video_info["Autor"]}'
                la duración es: '{video_info["Duracion"]}',
                la fecha de publicación es:'{video_info["Fecha_Publicacion"]}',
                el url es: '{video_info["URL"]}'
                y la transcripcion es:'{video_info["Transcript"]}'
                este resumen debe responder al tema del titulo
                y debe mostrar inicialmente los datos de titulo, 
                autor, duración, fecha de publicación y 
                url del transcript, seguidamente del resumen en menos de 100 palabras"""
        }],
        max_tokens=400)

        summary = respuesta.choices[0].message.content.strip()

        return summary

    except Exception as e:
        # En caso de error, devolver información incompleta con el error
        return {
            "Titulo": None,
            "Autor": None,
            "Duracion": None,
            "Fecha_Publicacion": None,
            "URL": youtube_url,
            "Transcript": f"Error: {e}",
        }

In [31]:
resumen = summary_video_yt("https://www.youtube.com/watch?v=DiXbQaDzB_c&ab_channel=LiveNOWfromFOX")

Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


[youtube] Extracting URL: https://www.youtube.com/watch?v=DiXbQaDzB_c&ab_channel=LiveNOWfromFOX
[youtube] DiXbQaDzB_c: Downloading webpage
[youtube] DiXbQaDzB_c: Downloading ios player API JSON
[youtube] DiXbQaDzB_c: Downloading mweb player API JSON
[youtube] DiXbQaDzB_c: Downloading player 6450230e


         player = https://www.youtube.com/s/player/6450230e/player_ias.vflset/en_US/base.js
         n = qZrpt0bKmtyIt6IW ; player = https://www.youtube.com/s/player/6450230e/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/6450230e/player_ias.vflset/en_US/base.js
         n = Iyi1HIaEk6jEIOSM ; player = https://www.youtube.com/s/player/6450230e/player_ias.vflset/en_US/base.js


[youtube] DiXbQaDzB_c: Downloading m3u8 information


In [32]:
import textwrap
parrafos = resumen.split('\n')
parrafos_formateados = [textwrap.fill(parrafo, width=60) for parrafo in parrafos]
texto_formateado = '\n'.join(parrafos_formateados)
print(texto_formateado)


**Título:** Full: President Trump takes questions in Oval
Office
**Autor:** LiveNOW from FOX
**Duración:** 1322
**Fecha de publicación:** 20250422
**URL:** [Ver video](https://www.youtube.com/watch?v=DiXbQaD
zB_c&ab_channel=LiveNOWfromFOX)

En esta conferencia, el presidente Trump presenta y
juramenta al nuevo presidente de la SEC, Paul Atkins,
destacando su experiencia en la regulación de mercados y su
compromiso con la innovación en criptomonedas. Atkins
enfatiza su enfoque en la protección de los inversores y la
transparencia en la regulación. Trump también aborda temas
de comercio internacional, la relación con China y los
avances en la economía estadounidense. Además, se discuten
cuestiones de seguridad nacional y políticas migratorias.
